In [2]:
import pandas as pd
import numpy as np

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
review_score_data = pd.read_csv('./dataset_csv/review_with_score.csv')
user_rank_data = pd.read_csv('./dataset_csv/user_ranking_new.csv')

In [4]:
review_score_data = review_score_data[['text', 'useful', 'business_id', 'cool', 'review_id', 'date', 'stars', 
                                          'funny', 'user_id', 'sentiment_score']]

In [5]:
users = []
for user in review_score_data['user_id']:
    user = user.split("'")[1]
    users.append(user)
    
review_score_data['user_id'] = users

In [6]:
merged_data = pd.merge(user_rank_data,review_score_data,on='user_id')

In [7]:
users = []
for user in merged_data['business_id']:
    user = user.split("'")[1]
    users.append(user)
    
merged_data['business_id'] = users

In [8]:
restaurant_data = pd.read_csv('./filtered_categories_csv/restaurants_food.csv', encoding = "ISO-8859-1")
shopping_data = pd.read_csv('./filtered_categories_csv/shopping.csv', encoding = "ISO-8859-1")
health_data = pd.read_csv('./filtered_categories_csv/healthnmedical.csv', encoding = "ISO-8859-1")
home_data = pd.read_csv('./filtered_categories_csv/homeservice.csv', encoding = "ISO-8859-1")
beauty_data = pd.read_csv('./filtered_categories_csv/beautynspa.csv', encoding = "ISO-8859-1")

In [9]:
print(merged_data.columns.values.tolist())
merged_cols = merged_data.columns.values.tolist()[1:]
print(merged_cols)

['Unnamed: 0', 'user_id', 'review_count', 'yelping_since', 'useful_x', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'text', 'useful_y', 'business_id', 'cool', 'review_id', 'date', 'stars', 'funny', 'sentiment_score']
['user_id', 'review_count', 'yelping_since', 'useful_x', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'text', 'useful_y', 'business_id', 'cool', 'review_id', 'date', 'stars', 'funny', 'sentiment_score']


In [10]:
merged_data = merged_data[merged_cols]

In [11]:
print(restaurant_data.shape, shopping_data.shape, health_data.shape, home_data.shape, beauty_data.shape)

(79395, 16) (27971, 16) (14230, 16) (16205, 16) (17014, 16)


In [12]:
def topUsersCategories(merged_data, category_data, business_category):
    final_merged = pd.merge(category_data, merged_data, on='business_id')
    final_merged.sort_values(['final_score', 'sentiment_score'], ascending=False, inplace=True)
    top_users = final_merged['user_id'].unique().tolist()[:10]
    business_type = [business_category] * 10
    top_user_data = pd.DataFrame({'user_id': top_users,
                                 'business_category': business_type})
    top_user_data.to_csv('./filtered_categories_csv/US_top_users.csv', mode='a', header=None)

In [22]:
topUsersCategories(merged_data, restaurant_data, "Restaurant")
topUsersCategories(merged_data, shopping_data, "Shopping")
topUsersCategories(merged_data, home_data, "Home & Service")
topUsersCategories(merged_data, health_data, "Health & Medical")
topUsersCategories(merged_data, beauty_data, "Beauty & Spa")

In [12]:
state_map = {'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Washington District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}

In [79]:
import itertools
def topUsersCategories(merged_data, category_data, business_category):
    final_merged = pd.merge(category_data, merged_data, on='business_id')
    final_merged.sort_values(['final_score'], ascending=False, inplace=True)
    for state in states:
        state_top_users = final_merged[final_merged['state'] == state]['user_id'].unique().tolist()[:10] 
        business_id = []
        for user in state_top_users:
            bid = final_merged[final_merged['user_id'] == user].sort_values(['sentiment_score'], 
                                                                                     ascending=False)['business_id'].tolist()[0]
            business_id.append(bid)
        """if len(business_id) < 10:
            business_id += [''] * (10 - len(business_id))"""
        business_type = [business_category] * 10
        state_name = list(state_map.keys())[list(state_map.values()).index(state)]
        state_list = [state_name] * 10
        state_abr = [state] * 10
        #print(len(business_id), len(business_type), len(state_list), len(state_top_users))
        final_list = [state_top_users, business_id, business_type, state_list, state_abr]
        state_data = pd.DataFrame((x for x in itertools.zip_longest(*final_list)), columns=['state_top_users', 'business_id', 
                                                                                            'business_type', 'state', 'state_abr'])
        state_data.to_csv('./filtered_categories_csv/US_top_statewise_users_new.csv', mode='a', header=None)

In [80]:
topUsersCategories(merged_data, restaurant_data, "Restaurant")
topUsersCategories(merged_data, shopping_data, "Shopping")
topUsersCategories(merged_data, home_data, "Home & Service")
topUsersCategories(merged_data, health_data, "Health & Medical")
topUsersCategories(merged_data, beauty_data, "Beauty & Spa")

In [11]:
us_state_data = pd.read_csv('./filtered_categories_csv/US_top_statewise_users.csv')
us_top_data = pd.read_csv('./filtered_categories_csv/US_top_users.csv')

In [24]:
user_main_data = pd.read_csv('./dataset_csv/user.csv')

In [13]:
list1 = us_state_data['user_id'].unique().tolist()
list2 = us_top_data['user_id'].unique().tolist()
total_users = list1 + list2
print(len(list1), len(list2), len(total_users))

339 26 365


In [25]:
user_ids = []
for user in user_main_data['user_id']:
    user = user.split("'")[1]
    user_ids.append(user)

user_main_data['user_id'] = user_ids

names = []
for name in user_main_data['name']:
    name = name.split("'")[1]
    names.append(name)
    
user_main_data['name'] = names

In [68]:
user_rank_data.sort_values(['final_score'], ascending=False, inplace=True)

In [69]:
all_users = user_rank_data[user_rank_data['user_id'].isin(total_users)]

In [70]:
all_users.shape

(338, 10)

In [71]:
col_list = all_users.columns.values.tolist()
print(all_users.columns.values.tolist())

['Unnamed: 0', 'user_id', 'review_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score']


In [72]:
col_list = col_list[1:]

In [73]:
all_users = all_users[col_list]

In [74]:
user_names = []
for user in all_users['user_id']:
    username = user_main_data[user_main_data['user_id'] == user]['name'].tolist()[0]
    user_names.append(username)
print(len(user_names))

338


In [75]:
all_users['name'] = user_names

In [79]:
all_users = all_users[['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'fans', 'elite_count', 
                       'compliment_count', 'months_since', 'final_score']]

In [80]:
all_users.to_csv('./filtered_categories_csv/US_all_users.csv')

In [26]:
business_json_data = None
with open('./dataset/business.json', encoding='utf-8') as data:
    business_json_data = data.readlines()
    
business_json_data = map(lambda x: x.rstrip(), business_json_data)

data_str = "[" + ",".join(business_json_data) + "]"

business_data = pd.read_json(data_str)

print(business_data.shape)

(174567, 15)


In [83]:
b_ids = us_state_data['business_id'].unique().tolist()

In [85]:
all_business = business_data[business_data['business_id'].isin(b_ids)]

In [88]:
all_business.shape

(360, 15)

In [89]:
all_business.to_csv('./filtered_categories_csv/US_all_business.csv')

In [12]:
from ast import literal_eval
cat_list_rest = []
for item in restaurant_data['categories']:
    item = literal_eval(item)
    cat_list_rest.append(item)
restaurant_data['categories'] = cat_list_rest

cat_list_shop = []
for item in shopping_data['categories']:
    item = literal_eval(item)
    cat_list_shop.append(item)
shopping_data['categories'] = cat_list_shop

cat_list_beauty = []
for item in beauty_data['categories']:
    item = literal_eval(item)
    cat_list_beauty.append(item)
beauty_data['categories'] = cat_list_beauty
    
cat_list_home = []
for item in home_data['categories']:
    item = literal_eval(item)
    cat_list_home.append(item)
home_data['categories'] = cat_list_home    

cat_list_health = []
for item in health_data['categories']:
    item = literal_eval(item)
    cat_list_health.append(item)
health_data['categories'] = cat_list_health

In [13]:
from collections import Counter

In [14]:
all_users = pd.read_csv('./filtered_categories_csv/US_all_users.csv')

In [15]:
users_list = all_users['user_id'].unique().tolist()

In [ ]:
temp = sum(restaurant_data.categories, [])
cate_count = Counter(temp)
most_common = cate_count.most_common(4)

In [18]:
print(most_common)

[('Restaurants', 64966), ('Food', 35125), ('Nightlife', 9068)]


In [16]:
def user_category_count(merged_data, category_data, category):
    final_merged = pd.merge(category_data, merged_data, on='business_id')
    index = 1
    sub_cat_food = "Food"
    for user in users_list:
        user_data = final_merged[final_merged['user_id'] == user]
        temp = sum(user_data.categories, [])
        cate_count = Counter(temp)
        main_common = cate_count.most_common(20)
        sub_cat1 = []
        sub_cat1_type = []
        if len(main_common) > 3:
            for x in main_common:
                if category == "Restaurants":
                    if (x[0] != "Food" and x[0] != "Restaurants") and (len(sub_cat1) != 3 and len(sub_cat1_type) != 3):
                        sub_cat1.append(x[0])
                        sub_cat1_type.append(x[1])
                else:
                    if x[0] != category and (len(sub_cat1) != 3 and len(sub_cat1_type) != 3):
                        sub_cat1.append(x[0])
                        sub_cat1_type.append(x[1])
            #print(user, sub_cat1, sub_cat1_type, len(main_common))
            #print(len(sub_cat1), len(sub_cat1_type))
            if len(sub_cat1) != 3 and len(sub_cat1_type) != 3 :
                sub_cat1 = sub_cat1 + [None]*(3-len(sub_cat1))
                sub_cat1_type = sub_cat1_type + [0]*(3-len(sub_cat1_type))
            cat_count_data = pd.DataFrame({'user_id': user, 'business_main_category': category,
                                          'sub_category1': sub_cat1[0], 
                                            'sub_category1_count': sub_cat1_type[0], 
                                          'sub_category2': sub_cat1[1], 
                                            'sub_category2_count': sub_cat1_type[1], 
                                          'sub_category3': sub_cat1[2], 
                                            'sub_category3_count': sub_cat1_type[2] }
                                                            , index=[index])
            cat_count_data.to_csv('./filtered_categories_csv/US_user_cat_count.csv', mode='a', header=None)
        index = index + 1

In [17]:
user_category_count(merged_data, restaurant_data, "Restaurants")
user_category_count(merged_data, shopping_data, "Shopping")
user_category_count(merged_data, home_data, "Home Services")
user_category_count(merged_data, health_data, "Health & Medical")
user_category_count(merged_data, beauty_data, "Beauty & Spas")

In [13]:
all_businesses = pd.read_csv('./filtered_categories_csv/US_all_business.csv')
map_data = pd.read_csv('./filtered_categories_csv/US_top_statewise_users_new.csv')

In [14]:
print(all_businesses.shape, map_data.shape)

(360, 16) (4881, 33)


In [20]:
map_data = map_data[['user_id', 'business_id', 'full_state', 'state']]

In [25]:
map_data.shape

(4881, 4)

In [30]:
joined_data = pd.merge(map_data, user_rank_data, how='left', on='user_id')
print(joined_data.shape)

(4881, 13)


In [31]:
joined_data.isnull().sum()

user_id             2022
business_id         2054
full_state             0
state                  0
Unnamed: 0          4356
review_count        4356
yelping_since       4356
useful              4356
fans                4356
elite_count         4356
compliment_count    4356
months_since        4356
final_score         4356
dtype: int64

In [27]:
print(len(map_data['business_id'].unique().tolist()))

362


In [28]:
merge_temp = pd.merge(map_data, all_businesses, on='business_id')

In [33]:
merge_final = pd.merge(merge_temp, all_users, on='user_id')

In [35]:
print(merge_final.columns.values.tolist())
final_merge_cols = merge_final.columns.values.tolist()

['Unnamed: 0_x', 'user_id', 'business_id', 'business_category', 'state_x', 'Unnamed: 0_y', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name_x', 'neighborhood', 'postal_code', 'review_count_x', 'stars', 'state_y', 'Unnamed: 0', 'name_y', 'review_count_y', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score']


In [36]:
final_merge_cols.remove('Unnamed: 0_x')
final_merge_cols.remove('Unnamed: 0_y')
final_merge_cols.remove('Unnamed: 0')

In [37]:
print(final_merge_cols)

['user_id', 'business_id', 'business_category', 'state_x', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name_x', 'neighborhood', 'postal_code', 'review_count_x', 'stars', 'state_y', 'name_y', 'review_count_y', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score']


In [38]:
merge_final = merge_final[final_merge_cols]

In [39]:
merge_final = merge_final.rename(columns={'state_x': 'full_state', 'name_x':'business_name',
                                         'review_count_x': 'business_review_count', 'state_y': 'state_small',
                                         'name_y': 'username', 'review_count_y': 'user_reviews_count'})

In [40]:
print(merge_final.columns.values.tolist())


['user_id', 'business_id', 'business_category', 'full_state', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'business_name', 'neighborhood', 'postal_code', 'business_review_count', 'stars', 'state_small', 'username', 'user_reviews_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score']


In [41]:
merge_final.to_csv('./filtered_categories_csv/US_combined_data.csv')

In [15]:
all_businesses = pd.read_csv('./filtered_categories_csv/US_all_business.csv')
map_data = pd.read_csv('./filtered_categories_csv/US_top_statewise_users_new.csv')

In [46]:
print(len(all_businesses[all_businesses['state'] == 'AL']))

1


In [50]:
map_data = map_data[['user_id', 'business_id', 'business_category', 'full_state']]
map_data.head()

,user_id,business_id,business_category,full_state
0,NaN,NaN,Restaurant,Alabama
1,NaN,NaN,Restaurant,Alabama
2,NaN,NaN,Restaurant,Alabama
3,NaN,NaN,Restaurant,Alabama
4,NaN,NaN,Restaurant,Alabama


In [20]:
user_score_data = pd.read_csv('./dataset_csv/user_ranking_new.csv')

In [23]:
print(user_score_data.shape, user_main_data.shape)

(1326100, 10) (1326101, 22)


In [34]:
user_cols = user_score_data.columns.values.tolist()
user_cols = user_cols[1:]
print(user_cols)
user_cols.append('name')

['user_id', 'review_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score']


In [26]:
user_merged = pd.merge(user_main_data, user_score_data, on="user_id")

In [36]:
print(user_merged.columns.values.tolist())

['compliment_hot', 'elite', 'useful_x', 'user_id', 'compliment_more', 'fans_x', 'compliment_note', 'compliment_profile', 'compliment_cool', 'average_stars', 'compliment_cute', 'compliment_funny', 'compliment_photos', 'compliment_list', 'review_count_x', 'compliment_writer', 'cool', 'friends', 'name', 'yelping_since_x', 'funny', 'compliment_plain', 'Unnamed: 0', 'review_count_y', 'yelping_since_y', 'useful_y', 'fans_y', 'elite_count', 'compliment_count', 'months_since', 'final_score']


In [37]:
user_merged = user_merged.rename(columns={'fans_y': 'fans', 'useful_y':'useful',
                                         'yelping_since_y': 'yelping_since', 'review_count_y': 'review_count'})

In [38]:
user_merged = user_merged[user_cols]

In [39]:
print(user_merged.columns.values.tolist())

['user_id', 'review_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'name']


In [40]:
user_merged = user_merged.sort_values(['final_score'], ascending=False)

In [51]:
map_merge = pd.merge(map_data, user_merged, on='user_id')
print(map_merge.shape)

(525, 13)


In [54]:
print(len(map_data['user_id'].unique().tolist()))

339


In [73]:
map_merge.head()

,user_id,business_id,business_category,full_state,review_count,yelping_since,useful,fans,elite_count,compliment_count,months_since,final_score,name
0,PcvbBOCOcs6_suRDH7TSTg,KCVnCb6TvTzoDp4t-Qf4wQ,Restaurant,Alaska,1071,2012-08-16,20586,159,2,5294,68,2.062390,Juan
1,H3KluKXf1mp3etCToCJhqg,j3csEfGzkwnXATdRoZDT-A,Restaurant,Alaska,399,2012-06-17,36,10,3,25,70,2.023055,Mike
2,Cr88dTMP_sfPX0KTIkoO9g,nYxa4JoHZ96kptIZhFlz9w,Restaurant,Alaska,2,2010-11-17,0,0,0,0,89,1.098765,Anthony
3,UA_PSxcBCxWIycTPcOxzTA,mD7zqv7Y3kvsa_p_MtTayg,Restaurant,Alaska,55,2011-05-13,0,1,0,0,83,1.025119,Trevor
4,o7g9FHCJ7Rfdwf3UdlB1Dg,0TgBr7wAMQOij6_JP9q3ag,Restaurant,Alaska,13,2012-04-09,1,0,0,0,72,0.888907,Whitney


In [74]:
final_map_merge = pd.merge(map_merge, business_data, on='business_id')

In [75]:
final_map_merge.shape

(524, 27)

In [76]:
final_map_merge = final_map_merge.rename(columns={'name_x':'user_name',
                                         'review_count_x': 'user_reviews_count',
                                         'name_y': 'business_name', 'review_count_y': 'business_review_count'})

In [77]:
final_map_merge.shape

(524, 27)

In [78]:
final_map_merge.to_csv('./filtered_categories_csv/US_combined_new.csv')

In [14]:
combined_data = pd.read_csv('./filtered_categories_csv/US_combined_new.csv')

In [15]:
combined_data.drop(labels=['full_state'], axis=1, inplace=True)

In [16]:
combined_data.shape

(524, 27)

In [17]:
combined_data = combined_data.drop_duplicates(keep=False)

In [18]:
combined_data.shape

(524, 27)

In [22]:
comb_cols = combined_data.columns.values.tolist()
print(comb_cols)
comb_cols = comb_cols[1:]

['Unnamed: 0', 'user_id', 'business_id', 'business_category', 'user_reviews_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'user_name', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'business_name', 'neighborhood', 'postal_code', 'business_review_count', 'stars', 'business_state']


In [23]:
combined_data = combined_data[comb_cols]

In [24]:
combined_data = combined_data.drop_duplicates(keep=False)

In [25]:
combined_data.shape

(436, 26)

In [26]:
print(combined_data.columns.values.tolist())

['user_id', 'business_id', 'business_category', 'user_reviews_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'user_name', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'business_name', 'neighborhood', 'postal_code', 'business_review_count', 'stars', 'business_state']


In [28]:
abr_states = combined_data['business_state'].tolist()

In [33]:
combined_data = combined_data[combined_data.business_state.isin(states)]

In [36]:
final_states = []
for s in combined_data['business_state']:
    state_name = list(state_map.keys())[list(state_map.values()).index(s)]
    final_states.append(state_name)
print(len(final_states))

424


In [37]:
combined_data['business_state'] = final_states

In [30]:
print(len(final_states), len(abr_states))

424 436


In [40]:
combined_data.to_csv('./filtered_categories_csv/final_csv/US_combined.csv')

In [62]:
import itertools
def topUsersCategoriesNew(merged_data, category_data, business_category):
    final_merged = pd.merge(category_data, merged_data, on='business_id')
    final_merged.sort_values(['final_score'], ascending=False, inplace=True)
    for state in states:
        state_top_users = final_merged[final_merged['state'] == state]['user_id'].unique().tolist()[:10] 
        business_id = []
        for user in state_top_users:
            business_data = final_merged[final_merged['user_id'] == user].sort_values(['sentiment_score'], 
                                                                                     ascending=False).iloc[:10, :]
            length = business_data.shape[0]
            business_data['business_category'] = [business_category] * length
            state_name = list(state_map.keys())[list(state_map.values()).index(state)]
            business_data['full_state'] = [state_name] * length
            business_data.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv', mode='a', header=None)

In [63]:
topUsersCategoriesNew(merged_data, restaurant_data, "Restaurants")
topUsersCategoriesNew(merged_data, shopping_data, "Shopping")
topUsersCategoriesNew(merged_data, home_data, "Home Services")
topUsersCategoriesNew(merged_data, health_data, "Health & Medical")
topUsersCategoriesNew(merged_data, beauty_data, "Beauty & Spas")

In [61]:
cols = ['address', 'attributes', 'business_id', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude',
           'name', 'neighborhood', 'postal_code', 'business_review_counts', 'business_stars', 'state', 'user_id', 
        'user_review_count', 'yelping_since', 'user_useful_count', 'fans', 'elite_count', 'compliment_count',
       'months_since', 'final_score', 'text', 'buisness_review_useful_count', 'cool', 'review_id', 'review_date',
       'review_stars', 'funny', 'sentiment_score', 'business_category', 'full_state']

In [66]:
US_merge_data = pd.read_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv', names=cols)

In [68]:
US_merge_data.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv')

In [69]:
US_merge_data = pd.read_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv')

In [72]:
names = []
for name in US_merge_data['review_date']:
    name = name.split("'")[1]
    names.append(name)
    
US_merge_data['review_date'] = names

In [74]:
new_text = []
for text in US_merge_data['text']:
    text = text.rstrip()
    text = text.replace("\\n", "")
    text = text.replace("\\", "")
    text = text[2:-1]
    new_text.append(text)

In [75]:
US_merge_data['text'] = new_text

In [76]:
US_merge_data.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv')

In [87]:
import itertools
def topUsersCategoriesNew2(merged_data, category_data, business_category):
    final_merged = pd.merge(category_data, merged_data, on='business_id')
    final_merged.sort_values(['final_score'], ascending=False, inplace=True)
    final_users = final_merged['user_id'].unique().tolist()[:10]
    for user in final_users:
        business_data = final_merged[(final_merged['user_id'] == user) & (final_merged['state'].isin(states))]\
        .sort_values(['sentiment_score'], 
                                        ascending=False).iloc[:10, :]
        length = business_data.shape[0]
        business_data['business_category'] = [business_category] * length
        business_data.to_csv('./filtered_categories_csv/final_csv/US_top_data.csv', mode='a', header=None)

In [88]:
topUsersCategoriesNew2(merged_data, restaurant_data, "Restaurants")
topUsersCategoriesNew2(merged_data, shopping_data, "Shopping")
topUsersCategoriesNew2(merged_data, home_data, "Home Services")
topUsersCategoriesNew2(merged_data, health_data, "Health & Medical")
topUsersCategoriesNew2(merged_data, beauty_data, "Beauty & Spas")

In [89]:
top_user_data = pd.read_csv('./filtered_categories_csv/US_top_users.csv')
all_top_data = pd.read_csv('./filtered_categories_csv/final_csv/US_top_data.csv', names=cols)

In [90]:
print(len(top_user_data['user_id'].unique().tolist()))
print(len(all_top_data['user_id'].unique().tolist()))

26
25


In [91]:
names = []
for name in all_top_data['review_date']:
    name = name.split("'")[1]
    names.append(name)
    
all_top_data['review_date'] = names

names = []
for name in all_top_data['review_id']:
    name = name.split("'")[1]
    names.append(name)
    
all_top_data['review_date'] = names

In [92]:
new_text = []
for text in all_top_data['text']:
    text = text.rstrip()
    text = text.replace("\\n", "")
    text = text.replace("\\", "")
    text = text[2:-1]
    new_text.append(text)
    
all_top_data['text'] = new_text

In [93]:
all_top_data.to_csv('./filtered_categories_csv/final_csv/US_top_users_data.csv')

In [125]:
list1 = top_user_data['user_id'].unique().tolist()
list2 = all_top_data['user_id'].unique().tolist()
import itertools
def topUsersCategoriesNew2(merged_data, category_data, business_category):
    final_merged = pd.merge(category_data, merged_data, on='business_id')
    final_merged.sort_values(['final_score'], ascending=False, inplace=True)
    for user in list1:
        business_data = final_merged[(final_merged['user_id'] == user) & (final_merged['state'].isin(states))]\
                                        .sort_values(['sentiment_score'], 
                                        ascending=False).iloc[:10, :]
        length = business_data.shape[0]
        business_data['business_category'] = [business_category] * length
        business_data.to_csv('./filtered_categories_csv/final_csv/US_top_data_new.csv', mode='a', header=None)

In [126]:
topUsersCategoriesNew2(merged_data, restaurant_data, "Restaurants")
topUsersCategoriesNew2(merged_data, shopping_data, "Shopping")
topUsersCategoriesNew2(merged_data, home_data, "Home Services")
topUsersCategoriesNew2(merged_data, health_data, "Health & Medical")
topUsersCategoriesNew2(merged_data, beauty_data, "Beauty & Spas")

In [99]:
US_combined = pd.read_csv('./filtered_categories_csv/final_csv/US_combined.csv')

In [101]:
US_combined_cols = US_combined.columns.values.tolist()[1:]
US_combined = US_combined[US_combined_cols]

In [103]:
US_combined.drop(labels=['is_open', 'neighborhood', 'hours', 'city'], axis=1, inplace=True)

In [104]:
US_combined.drop_duplicates(inplace=True, keep=False)

In [105]:
US_combined.to_csv('./filtered_categories_csv/final_csv/US_combined_new.csv')

In [146]:
US_statewise = pd.read_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv')

In [108]:
US_statewise_cols = US_statewise.columns.values.tolist()[2:]
US_statewise = US_statewise[US_statewise_cols]

In [110]:
US_statewise.drop(labels=['is_open', 'neighborhood', 'hours', 'city'], axis=1, inplace=True)

In [111]:
US_statewise.drop_duplicates(inplace=True, keep=False)

In [112]:
US_statewise.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data_new.csv')

In [113]:
print(US_statewise.shape)
US_statewise = US_statewise[US_statewise['state'].isin(states)]
print(US_statewise.shape)

(2129, 30)
(1967, 30)


In [114]:
US_statewise.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data_new.csv')

In [127]:
US_top_user = pd.read_csv('./filtered_categories_csv/final_csv/US_top_data_new.csv', names=cols)

In [129]:
US_top_user.drop(labels=['full_state'], axis=1, inplace=True)

In [130]:
US_top_user.drop_duplicates(inplace=True, keep=False)

In [131]:
names = []
for name in US_top_user['review_date']:
    name = name.split("'")[1]
    names.append(name)
    
US_top_user['review_date'] = names

names = []
for name in US_top_user['review_id']:
    name = name.split("'")[1]
    names.append(name)
    
US_top_user['review_id'] = names

new_text = []
for text in US_top_user['text']:
    text = text.rstrip()
    text = text.replace("\\n", "")
    text = text.replace("\\", "")
    text = text[2:-1]
    new_text.append(text)
    
US_top_user['text'] = new_text

In [132]:
print(len(US_top_user['state'].tolist()))
past = US_top_user['state'].tolist()
new_state = []
for s in US_top_user['state']:
    state_name = list(state_map.keys())[list(state_map.values()).index(s)]
    new_state.append(state_name)
print(len(new_state))

428
428


In [134]:
US_top_user['state'] = new_state

In [135]:
US_top_user.to_csv('./filtered_categories_csv/final_csv/US_top_data_new.csv')

In [121]:
US_top_user = US_top_user[US_top_user['state'].isin(states)]

In [122]:
US_top_user.to_csv('./filtered_categories_csv/final_csv/US_top_data_new.csv')

In [147]:
US_statewise = pd.read_csv('./filtered_categories_csv/final_csv/US_top_statewise_data.csv')

In [148]:
US_statewise_cols = US_statewise.columns.values.tolist()[2:]
US_statewise = US_statewise[US_statewise_cols]

In [149]:
US_statewise.drop(labels=['full_state'], axis=1, inplace=True)

In [150]:
US_statewise = US_statewise[US_statewise['state'].isin(states)]

In [151]:
print(len(US_statewise['state'].tolist()))
past = US_statewise['state'].tolist()
new_state = []
for s in US_statewise['state']:
    state_name = list(state_map.keys())[list(state_map.values()).index(s)]
    new_state.append(state_name)
print(len(new_state))

2109
2109


In [152]:
print(past[0])
print(new_state[0])

AZ
Arizona


In [153]:
US_statewise['state'] = new_state

In [154]:
print(US_statewise.shape)
US_statewise.drop_duplicates(inplace=True, keep=False)
print(US_statewise.shape)

(2109, 33)
(1462, 33)


In [156]:
US_statewise.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data_new.csv')

In [157]:
US_statewise = pd.read_csv('./filtered_categories_csv/final_csv/US_top_statewise_data_new.csv')

In [159]:
US_statewise_cols = US_statewise.columns.values.tolist()[2:]
US_statewise = US_statewise[US_statewise_cols]

In [160]:
US_statewise.shape

(1462, 32)

In [161]:
US_statewise.drop(labels=['is_open', 'neighborhood', 'hours', 'city'], axis=1, inplace=True)

In [162]:
print(US_statewise.shape)
US_statewise.drop_duplicates(inplace=True, keep=False)
print(US_statewise.shape)

(1462, 28)
(1462, 28)


In [163]:
US_statewise.to_csv('./filtered_categories_csv/final_csv/US_top_statewise_data_new.csv')

In [164]:
top_users = pd.read_csv('./filtered_categories_csv/US_top_users.csv')
top_users.head()

,Unnamed: 0,business_category,user_id
0,0,Restaurant,6s-g2vFu12OemhiK3FJuOQ
1,1,Restaurant,JjXuiru1_ONzDkYVrHN0aw
2,2,Restaurant,VHdY6oG2JPVNjihWhOooAQ
3,3,Restaurant,nkN_do3fJ9xekchVC-v68A
4,4,Restaurant,om5ZiponkpRqUNa3pVPiRg


In [165]:
user_rank_data.shape

(1326100, 10)

In [166]:
top_users.shape

(50, 3)

In [167]:
merge_top = pd.merge(top_users, user_rank_data, on='user_id') 

In [168]:
merge_top.shape

(50, 12)

In [172]:
names = []
for usr in merge_top['user_id']:
    usr = user_main_data[user_main_data['user_id'] == usr]['name'].tolist()[0]
    names.append(usr)
    
print(len(names))

50


In [173]:
merge_top['username'] = names

In [174]:
merge_cols = merge_top.columns.values.tolist()

In [175]:
print(merge_cols)

['Unnamed: 0_x', 'business_category', 'user_id', 'Unnamed: 0_y', 'review_count', 'yelping_since', 'useful', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'username']


In [176]:
merge_cols.remove('Unnamed: 0_x')
merge_cols.remove('Unnamed: 0_y')

In [177]:
merge_top = merge_top[merge_cols]

In [179]:
merge_top.to_csv('./filtered_categories_csv/final_csv/US_top_full_user_data.csv')

In [180]:
US_state = pd.read_csv('./filtered_categories_csv/final_csv/US_top_statewise_data_new.csv')

In [181]:
US_state.shape

(1462, 29)

In [12]:
new_map_data = pd.read_csv('./filtered_categories_csv/US_top_statewise_mapping.csv')

In [13]:
new_map_data.shape

(4881, 19)

In [14]:
new_map_data.dropna(inplace=True, axis=0)

In [15]:
new_map_data.shape

(0, 19)

In [13]:
import itertools
def topUsersCategories(merged_data, category_data, business_category):
    
    final_merged = pd.merge(merged_data, category_data, on='business_id')
    final_merged.sort_values(['final_score'], ascending=False, inplace=True)
    
    for state in states:
        #print("For the state: ",state)
        #print("Values for this state",len(final_merged[final_merged['state'] == state]))
        state_top_users = final_merged[final_merged['state'] == state]['user_id'].unique().tolist()[:10] 
        #print("length of 10 users of state: ",len(state_top_users),"the users are: ",state_top_users)
        
        for user in state_top_users:
            
            #print("Inside the second loop: ")
            
            state_data = final_merged[(final_merged['user_id'] == user) & ((final_merged['state'] == state)) ].sort_values(['sentiment_score'], 
                                                                                     ascending=False).head(10)
            
            #print("Data for each user= ",user," length: ",len(state_data))
            length = state_data.shape[0]
            business_type = [business_category] * length
            state_name = list(state_map.keys())[list(state_map.values()).index(state)]
            #print("State for this user: ",state_name)
            #print(state_data['state'].head(1))
            state_list = [state_name] * length
            state_data['business_category'] = business_type
            state_data['full_state'] = state_list
            state_data.to_csv('./filtered_categories_csv/final_csv/state_data.csv', mode='a', header=None)
       
        
       

In [14]:
topUsersCategories(merged_data, restaurant_data, "Restaurant")
topUsersCategories(merged_data, shopping_data, "Shopping")
topUsersCategories(merged_data, home_data, "Home & Service")
topUsersCategories(merged_data, health_data, "Health & Medical")
topUsersCategories(merged_data, beauty_data, "Beauty & Spa")

In [44]:
print(['user_id', 'review_count_x', 'yelping_since', 'useful_x', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'text', 'useful_y', 'business_id', 'cool', 'review_id', 'date', 'stars_x', 'funny', 'sentiment_score', 'Unnamed: 0', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood', 'postal_code', 'review_count_y', 'stars_y', 'state', 'business_category', 'full_state'])

['user_id', 'review_count_x', 'yelping_since', 'useful_x', 'fans', 'elite_count', 'compliment_count', 'months_since', 'final_score', 'text', 'useful_y', 'business_id', 'cool', 'review_id', 'date', 'stars_x', 'funny', 'sentiment_score', 'Unnamed: 0', 'address', 'attributes', 'categories', 'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood', 'postal_code', 'review_count_y', 'stars_y', 'state', 'business_category', 'full_state']


In [15]:
cols_new = ['user_id', 'review_count_x', 'yelping_since', 'useful_x', 'fans', 'elite_count', 'compliment_count', 
            'months_since', 'final_score', 'text', 'useful_y', 'business_id', 'cool', 'review_id', 'date', 
            'stars_x', 'funny', 'sentiment_score', 'Unnamed: 0', 'address', 'attributes', 'categories', 
            'city', 'hours', 'is_open', 'latitude', 'longitude', 'name', 'neighborhood', 'postal_code', 
            'review_count_y', 'stars_y', 'state', 'business_category', 'full_state']

In [20]:
combined_new = combined_new.rename(columns={'useful_x':'user_useful_review_count', 'useful_y':'business_useful_review_count',
                                         'review_count_x': 'user_reviews_count', 'stars_x': 'business_stars',
                                         'stars_y': 'business_review_stars', 'review_count_y': 'business_review_count'})

In [16]:
combined_new = pd.read_csv('./filtered_categories_csv/final_csv/state_data.csv', names=cols_new)

In [17]:
cols_new.remove('Unnamed: 0')
combined_new = combined_new[cols_new]

In [21]:
names = []
for name in combined_new['date']:
    name = name.split("'")[1]
    names.append(name)
    
combined_new['date'] = names

names = []
for name in combined_new['review_id']:
    name = name.split("'")[1]
    names.append(name)
    
combined_new['review_id'] = names

new_text = []
for text in combined_new['text']:
    text = text.rstrip()
    text = text.replace("\\n", "")
    text = text.replace("\\", "")
    text = text[2:-1]
    new_text.append(text)
    
combined_new['text'] = new_text

In [22]:
print(len(combined_new['user_id'].tolist()))

1310


0    Johnny
1     Chris
2     Tiffy
3      Mark
4    Evelyn
Name: name, dtype: object

In [26]:
usernames = []
for user in combined_new['user_id']:
    name = user_main_data[user_main_data['user_id'] == user]['name'].tolist()[0]
    print(name)
    usernames.append(name)
    
print(len(usernames))

Juan
Mike
Anthony
Trevor
Whitney
Benjamin
Richard
Linda
Matt
S
Jeremy
Jeremy
Jeremy
Jeremy
Jeremy
Jeremy
Jeremy
Jeremy
Andrea
Andrea
Andrea
Andrea
Andrea
Andrea
Andrea
Andrea
Andrea
Andrea
Miriam
Ruggy
Ruggy
Nish
Nish
Nish
Nish
Nish
Nish
Jando
Jando
Jando
Jando
Jando
Jando
Jando
Jando
Colleen
Colleen
Colleen
Colleen
Colleen
Colleen
Colleen
Ed
Ed
Ed
Ed
Ed
Ed
Ed
Ed
Ed
Ed
Peter
Peter
Peter
Peter
Peter
Peter
Peter
Peter
Peter
Geoff
Mariaroberta
Claudio
Maria
Giuseppe
Rebecca
Megan
Carrie
Ger
Anne
Sarah
Jeremy
Jeremy
Jeremy
Jeremy
Jeremy
Ellen
Ellen
Ellen
Ellen
Ellen
Ellen
Ellen
Ellen
Ellen
Ellen
Jacob
Kyle
Shannon
Shannon
Neal
Neal
Neal
Mike
Mike
Melanie
Melanie
Nancy
Stephen
Joel
Melissa
Constancia
Brian
Brian
Leslie
Matt
John
Jeff
Jenn
Rob
Rob
Dave
Dave
Dave
Dave
Dave
Dave
Dave
Dave
Dave
Dave
Jessica
Jessica
Jessica
Jeremy
Jeremy
Jeremy
Jeremy
Jeremy
Andrea
Andrea
Miriam
Miriam
Miriam
Miriam
Miriam
Miriam
Miriam
Miriam
Miriam
Miriam
Ruggy
Nish
Nish
Nish
Nish
Nish
Nish
Jando
Jando
Jando
J

Rebecca
Rebecca
Rebecca
Stephanie
Billy
Billy
Melissa
Aisha
1310


In [27]:
usernames[0]

'Juan'

In [28]:
combined_new['username'] = usernames

In [29]:
combined_new.to_csv('./filtered_categories_csv/final_csv/US_combined_new.csv')